In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import tensorflow as tf


import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from urllib.request import urlopen,urlretrieve

from tensorflow.keras.models import load_model
from sklearn.datasets import load_files   
#from tensorflow.keras.utils import np_utils
from glob import glob
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint

class_names = sorted(os.listdir(r"/home/nvme/data/train/train"))

N_classes = len(class_names)

In [2]:
base_model = tf.keras.applications.resnet.ResNet101(
    input_shape = (224,224,3),
    include_top = False)

In [3]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
outputs = Dense(N_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = outputs)

In [4]:
from tensorflow.keras.optimizers import SGD, Adam

adam = Adam(lr = 0.0001)
model.compile(optimizer = adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# example of progressively loading images from file
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale = 1./255.,
    horizontal_flip=True,
    validation_split=0.2)

train_gen = datagen.flow_from_directory('/home/nvme/data/train/train',
                                        class_mode = "categorical",
                                        target_size = (224, 224),
                                        batch_size = 32,
                                        shuffle = True,
                                        subset = "training",
                                        seed = 42)
val_gen = datagen.flow_from_directory('/home/nvme/data/train/train',
                                        class_mode = "categorical",
                                        target_size = (224, 224),
                                        batch_size = 32,
                                        shuffle = True,
                                        subset = "validation",
                                        seed = 42)

Found 22443 images belonging to 17 classes.
Found 5602 images belonging to 17 classes.


In [6]:
import datetime

#%reload_ext tensorboard

#log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


epochs = 2
# fits the model on batches with real-time data augmentation:
model.fit_generator(train_gen,
    steps_per_epoch = train_gen.samples // 32,
    validation_data = val_gen, 
    validation_steps = val_gen.samples // 32,
    epochs = epochs 
    #,callbacks=[tensorboard_callback]
                   )

#model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
#del model

Epoch 1/3
701/701 [==============================] - 717s 1s/step - loss: 0.6135 - accuracy: 0.8132 - val_loss: 0.3975 - val_accuracy: 0.8723
Epoch 2/3
701/701 [==============================] - 712s 1s/step - loss: 0.3876 - accuracy: 0.8739 - val_loss: 0.3782 - val_accuracy: 0.8748
Epoch 3/3
701/701 [==============================] - 710s 1s/step - loss: 0.3280 - accuracy: 0.8939 - val_loss: 0.3669 - val_accuracy: 0.8820


In [7]:
from tensorflow.keras.models import load_model

#model = load_model('resnet101.h5')

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_test = ImageDataGenerator(rescale = 1./255.)

test_gen = datagen_test.flow_from_directory('/home/nvme/data/test',
                                        #class_mode = "categorical",
                                        target_size = (224, 224),
                                        batch_size = 1,
                                        shuffle = False)

pred = model.predict_generator(test_gen, verbose = 1)

Found 7958 images belonging to 1 classes.
7958/7958 [==============================] - 789s 99ms/step


In [8]:
import numpy as np
p = np.argmax(pred, axis = 1)
predictions = [class_names[k] for k in p]
a = np.arange(len(predictions))
d = {'Id': a, 'Category': predictions}

import pandas as pd
df = pd.DataFrame(d)

df.to_csv("submission.csv", index = None, header = True)

In [10]:
df.head(20)

,Id,Category
0,0,Boat
1,1,Van
2,2,Truck
3,3,Van
4,4,Bicycle
5,5,Truck
6,6,Car
7,7,Boat
8,8,Tank
9,9,Car
